In [1]:
# Parameters
cpu = 8
group_name = "DG_Glut"
mem_gb = 10


In [2]:
import pandas as pd
import subprocess
import numpy as np

In [3]:
#group_name = 'DG_Glut'

In [4]:
home_dir = '/home/qzeng_salk_edu/project/241018_pre_ml'
ct = group_name

dmr_dir = '/data/mC_FILTER_DMR_BED'
_type = 'Total'

In [5]:
dmr_bf = pd.read_csv(f'{dmr_dir}/{ct}.{_type}.aDMR.bed', sep='\t', header=None)
dmr_bf.head()

,0,1,2,3,4,5,6
0,chr1,6846251,6846299,chr1-6846251-6846299,0.81,0.68,0.56
1,chr1,9169723,9169728,chr1-9169723-9169728,0.76,0.60,0.51
2,chr1,10148905,10148983,chr1-10148905-10148983,0.73,0.58,0.53
3,chr1,10164948,10165945,chr1-10164948-10165945,0.53,0.35,0.20
4,chr1,12806641,12806697,chr1-12806641-12806697,0.26,0.41,0.48


In [6]:
# gene_df=  pd.read_csv('/data/ref/mouse/genome_anno/mm10_gene.bed', sep='\t', header=None)
# # extend 1 and 2 2000kb
# gene_df[1] = gene_df[1] - 2000
# gene_df[2] = gene_df[2] + 2000
# #gene_df.to_csv('mm10_gene_2kb.bed', sep='\t', header=None, index=None) 
# gene_df.head()

In [7]:
# use bedtools intersect to find the genes that overlap with the DMRs
command = f"bedtools intersect -a {dmr_dir}/{ct}.{_type}.aDMR.bed -b {home_dir}/mm10_gene_2kb.bed -wa -wb > {ct}.aDMR_gene.bed"
subprocess.run(command, shell=True)

CompletedProcess(args='bedtools intersect -a /data/mC_FILTER_DMR_BED/DG_Glut.Total.aDMR.bed -b /home/qzeng_salk_edu/project/241018_pre_ml/mm10_gene_2kb.bed -wa -wb > DG_Glut.aDMR_gene.bed', returncode=0)

In [8]:
dmr_gene_df = pd.read_csv(f'{ct}.aDMR_gene.bed', sep='\t', header=None)
# add a columns that is np.log2([6] - [4])
dmr_gene_df.columns = ['chr','start','end','dmr_id','2mo','9mo','18mo', 
                        'gene_chr','gene_start','gene_end','gene_id','gene_strand','gene_name','gene_type']
dmr_gene_df['old-young'] = dmr_gene_df['18mo'] -dmr_gene_df['2mo']
dmr_gene_df.head()

,chr,start,end,dmr_id,2mo,9mo,18mo,gene_chr,gene_start,gene_end,gene_id,gene_strand,gene_name,gene_type,old-young
0,chr1,6846251,6846299,chr1-6846251-6846299,0.81,0.68,0.56,chr1,6485231,6862940,ENSMUSG00000033740.13,+,St18,protein_coding,-0.25
1,chr1,9169723,9169728,chr1-9169723-9169728,0.76,0.60,0.51,chr1,8359475,9301878,ENSMUSG00000025909.12,-,Sntg1,protein_coding,-0.25
2,chr1,10148905,10148983,chr1-10148905-10148983,0.73,0.58,0.53,chr1,10135571,10234670,ENSMUSG00000067851.7,-,Arfgef1,protein_coding,-0.20
3,chr1,10164948,10165945,chr1-10164948-10165945,0.53,0.35,0.20,chr1,10164539,10168778,ENSMUSG00000088153.1,+,Gm26348,misc_RNA,-0.33
4,chr1,10164948,10165945,chr1-10164948-10165945,0.53,0.35,0.20,chr1,10135571,10234670,ENSMUSG00000067851.7,-,Arfgef1,protein_coding,-0.33


In [9]:

subprocess.run(f"rm {ct}.aDMR_gene.bed", shell=True)

CompletedProcess(args='rm DG_Glut.aDMR_gene.bed', returncode=0)

In [10]:
# drop dmr_id, gen_id, gene_strand, old-young
dmr_gene_df['distance'] = np.minimum(abs(dmr_gene_df['start'] - dmr_gene_df['gene_start']), abs(dmr_gene_df['start'] - dmr_gene_df['gene_end']))
dmr_gene_df = dmr_gene_df.drop(columns=['chr','gene_chr','dmr_id','gene_id','gene_strand','old-young','gene_type'])
dmr_gene_df.head()

,start,end,2mo,9mo,18mo,gene_start,gene_end,gene_name,distance
0,6846251,6846299,0.81,0.68,0.56,6485231,6862940,St18,16689
1,9169723,9169728,0.76,0.60,0.51,8359475,9301878,Sntg1,132155
2,10148905,10148983,0.73,0.58,0.53,10135571,10234670,Arfgef1,13334
3,10164948,10165945,0.53,0.35,0.20,10164539,10168778,Gm26348,409
4,10164948,10165945,0.53,0.35,0.20,10135571,10234670,Arfgef1,29377


In [11]:
dmr_gene_df.to_csv(f'{home_dir}/ml_input/{ct}/{ct}.aDMR_gene.csv', index=None)

## load enhancer

In [12]:
mc_abc_dir = '/data/female-amb/ABC.Enhancer/Re.ABC.By.Methylation'

In [13]:
mouse_genes = pd.read_csv(f"/home/qzeng_salk_edu/project/240321_abc_score/use.mouse_genes.csv", index_col =0)

In [14]:
def load_abc_df(ct, age):
    df = pd.read_csv(f'{mc_abc_dir}/{ct}/filtered.{ct}.{age}.abc_score.csv', index_col =0)
    if age == '8wk':
        age = '2mo'
    df.columns =  [f'{age}.activity',f'{age}.contact',f'{age}.abc_score']
    return df

In [15]:
young_abc_df = load_abc_df(ct, '8wk')
middle_abc_df = load_abc_df(ct, '9mo')
old_abc_df = load_abc_df(ct, '18mo')

In [16]:
# concat the three age groups
abc_df = pd.concat([young_abc_df, middle_abc_df, old_abc_df], axis=1)
#abc_df = abc_df.dropna()#
#abc_df = abc_df.fillna(0)

In [17]:
abc_df['chr'] = abc_df.index.map(lambda x: x.split('-')[0])
abc_df['start'] = abc_df.index.map(lambda x: int(x.split('-')[1]))
abc_df['end'] = abc_df.index.map(lambda x: int(x.split('-')[2]))
abc_df['gene_id'] = abc_df.index.map(lambda x: x.split('-')[3])
abc_df['gene_name'] = abc_df['gene_id'].map(lambda x: mouse_genes.loc[x,'gene_name'])

abc_df['gene_start'] = abc_df['gene_id'].map(lambda x: mouse_genes.loc[x,'start'])
abc_df['gene_end'] = abc_df['gene_id'].map(lambda x: mouse_genes.loc[x,'end'])
#abc_df['gene_length'] = abc_df['gene_end'] - abc_df['gene_start']
#abc_df['contact_distance'] = abs(abc_df['end'] - abc_df['start'])
# add eg_distance, based on the start is more close to gene_start ot gene_end; use the smaller absolute value
#abc_df['eg_distance'] = np.minimum(abs(abc_df['start'] - abc_df['gene_start']), abs(abc_df['start'] - abc_df['gene_end']))
#abc_df drop chr, start, end, gene_start, gene_end
#abc_df = abc_df.drop(columns=['chr','start','end','gene_start','gene_end'])
abc_df


,2mo.activity,2mo.contact,2mo.abc_score,9mo.activity,9mo.contact,9mo.abc_score,18mo.activity,18mo.contact,18mo.abc_score,chr,start,end,gene_id,gene_name,gene_start,gene_end
chr1-3681290-3681410-ENSMUSG00000051951,0.807107,65.0,0.069262,0.775056,51.0,0.055937,0.831209,109.0,0.088025,chr1,3681290,3681410,ENSMUSG00000051951,Xkr4,3205901,3671498
chr1-3703215-3703791-ENSMUSG00000051951,0.637767,33.0,0.027786,0.615269,31.0,0.026991,0.598799,35.0,0.020362,chr1,3703215,3703791,ENSMUSG00000051951,Xkr4,3205901,3671498
chr1-3524163-3524482-ENSMUSG00000089699,0.676796,35.0,0.040300,0.706448,57.0,0.065909,0.658778,88.0,0.074832,chr1,3524163,3524482,ENSMUSG00000089699,Gm1992,3466587,3513553
chr1-3540806-3540973-ENSMUSG00000089699,0.736671,36.0,0.045118,0.705843,43.0,0.049678,0.720259,63.0,0.058573,chr1,3540806,3540973,ENSMUSG00000089699,Gm1992,3466587,3513553
chr1-3668112-3668782-ENSMUSG00000089699,0.846691,22.0,0.031690,NaN,NaN,NaN,NaN,NaN,NaN,chr1,3668112,3668782,ENSMUSG00000089699,Gm1992,3466587,3513553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chr7-6903472-6903796-ENSMUSG00000108113,NaN,NaN,NaN,NaN,NaN,NaN,0.912230,32.0,0.021937,chr7,6903472,6903796,ENSMUSG00000108113,Zim2,6651748,6662368
chrX-42399128-42399257-ENSMUSG00000105993,NaN,NaN,NaN,NaN,NaN,NaN,0.799740,22.0,0.020807,chrX,42399128,42399257,ENSMUSG00000105993,Tex13c2,42489691,42491524
chrX-100530005-100530277-ENSMUSG00000044359,NaN,NaN,NaN,NaN,NaN,NaN,0.567922,66.0,0.025216,chrX,100530005,100530277,ENSMUSG00000044359,P2ry4,100590154,100594869
chrX-100531982-100532386-ENSMUSG00000044359,NaN,NaN,NaN,NaN,NaN,NaN,0.535110,66.0,0.023759,chrX,100531982,100532386,ENSMUSG00000044359,P2ry4,100590154,100594869


In [18]:
abc_df.to_csv(f'{home_dir}/ml_input/{ct}/{ct}.abc_enhancer.DMR_gene.csv', index=None)